# L2: Image captioning app 🖼️📝

Load your HF API key and relevant Python libraries

In [1]:
import os
import io
import IPython.display
from utils import *
from PIL import Image
import base64 
import gradio as gr

# Helper functions
import requests, json
from dotenv import load_dotenv, find_dotenv

# Get the base project path and data path
BASE_PATH = os.environ['BASE_PATH']
DATA_PATH = os.path.join(BASE_PATH, "data")

## Building an image captioning app 

Here we'll be using an [Inference Endpoint](https://huggingface.co/inference-endpoints) for `Salesforce/blip-image-captioning-base` a 14M parameter captioning model.

The free images are available on: https://free-images.com/

In [2]:
image_url = "https://free-images.com/sm/9596/dog_animal_greyhound_983023.jpg"
display(IPython.display.Image(url=image_url))
result = get_image_caption_from_url(image_url)
print(result)

[{'generated_text': 'a dog wearing a santa hat and a red scarf'}]


## Captioning with `gr.Interface()`

#### gr.Image()
- The `type` parameter is the format that the `fn` function expects to receive as its input.  If `type` is `numpy` or `pil`, `gr.Image()` will convert the uploaded file to this format before sending it to the `fn` function.
- If `type` is `filepath`, `gr.Image()` will temporarily store the image and provide a string path to that image location as input to the `fn` function.

In [3]:
# Construct paths to the image files
dog_path = os.path.join(DATA_PATH, "christmas_dog.jpeg")
bird_path = os.path.join(DATA_PATH, "bird_flight.jpeg")
cow_path = os.path.join(DATA_PATH, "cow.jpeg")

In [6]:
demo = gr.Interface(fn=captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    description="Caption any image using the BLIP model",
                    flagging_mode="never",
                    examples=[dog_path, bird_path, cow_path])

demo.launch(share=True, allowed_paths=[DATA_PATH], server_port=int(os.environ['PORT1']))

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://177aeea4684fe8bb29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
gr.close_all()

Closing server running on port: 7860
Closing server running on port: 7860
